# Routing Topology: OSPF using FRRouting

This notebook is an example of how to create a FABRIC routing experiment topology comprising nodes at three different sites. Each site has a local layer 2 (Ethernet) network connecting a set of local nodes and one gateway router. The three gateway routers connect to each other and use the [FRRouting](https://frrouting.org/) protocol suite to deploy [OSPF](https://en.wikipedia.org/wiki/Open_Shortest_Path_First) dameons to propagate route updates across the topology.

You might be familiar with the [Quagga](https://www.quagga.net/) router suite.  FRRouting is based on Quagga but has a more active upstream community including many large companies working on cloud networking.


## Step 1:  Configure the Environment


In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join(f"{os.environ['HOME']}/work/PRUTH-FABRIC-Examples/fablib_local"))
if module_path not in sys.path:
    sys.path.append(module_path)
from fablib_custom.fablib_custom import *

from chameleon_utils.chameleon_config import *
load_chameleon_rc_environment(chameleon_rc_file=f"{os.environ['HOME']}/work/fablib_local_private_config/Chameleon-openrc.sh")


from chameleon_utils.chameleon_stitching import *
from chameleon_utils.chameleon_servers import *

from fablib_common_utils.utils import *

from performance_testing.iperf3 import *

from my_experiment.frrouting_experiment import FRRouting_Experiment

## Step 2: Import the FABLlib Library


In [ ]:
frr_experiment = FRRouting_Experiment('FRRouting_OSPF',
                             output_type='HTML',
                             node_tools=f"{os.environ['HOME']}/work/PRUTH-FABRIC-Examples/fabric_node_tools")

## Step 3 (Optional): Query for Available Tesbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [ ]:
try:
    frr_experiment.list_resources()

except Exception as e:
    print(f"Exception: {e}")

## Step 4: Create the Experiment Slice

The following creates private layer 2 networks on three sites including a OSPF gateway routers that propogate routes acrross the topology. 


In [ ]:
#slice_name = 'OSPF_Routing_Topology1'

time_stamp = datetime.now(tz=tz.tzutc()).strftime('%Y%m%d%H%M')

ext=f'_pruth_{time_stamp}'

#[site1,site2,site3] = fablib.get_random_sites(count=3)
#[site1,site2,site3] = ['UTAH','SALT','STAR']
[site1,site2,site3] = ['DALL','SALT','STAR']

print(f"Sites: {site1},{site2},{site3}")

router_base_name='router'
router_link_base_name='router_link'

node_base_name='node'
local_network_base_name='net_local'

site_node_count=2


router1_name = f"{router_base_name}1"
router2_name = f"{router_base_name}2"
router3_name = f"{router_base_name}3"

#Create Router Links
route_link1_name = f'{router_link_base_name}1'
route_link2_name = f'{router_link_base_name}2'
route_link3_name = f'{router_link_base_name}3'

#Create Site Local Networks
site1_local_net_name = f'{local_network_base_name}1'
site2_local_net_name = f'{local_network_base_name}2'
site3_local_net_name = f'{local_network_base_name}3'
    


In [ ]:
import traceback
try:
    #frr_slice = FRRouting_Slice(slice_name)
    

    router1 = frr_experiment.add_router(name=f'{site1}_Router', site=site1, cores=32, ram=128, disk=10)
    router2 = frr_experiment.add_router(name=f'{site2}_Router', site=site2, cores=32, ram=128, disk=10)
    router3 = frr_experiment.add_router(name=f'{site3}_Router', site=site3, cores=32, ram=128, disk=10)
    
    
    frr_experiment.add_router_link(name=f'{router1.get_site()}_to_{router2.get_site()}', router1=router1, router2=router2, nic_model='NIC_Basic')
    frr_experiment.add_router_link(name=f'{router2.get_site()}_to_{router3.get_site()}', router1=router2, router2=router3, nic_model='NIC_Basic')
    frr_experiment.add_router_link(name=f'{router3.get_site()}_to_{router1.get_site()}', router1=router3, router2=router1, nic_model='NIC_Basic')

    frr_experiment.add_local_network(name=f'{router1.get_site()}_local_net', router=router1, node_count=1, cores=32, ram=128, disk=10)
    frr_experiment.add_local_network(name=f'{router2.get_site()}_local_net', router=router2, node_count=1, cores=32, ram=128, disk=10)
    #frr_experiment.add_local_network(name=f'{router3.get_site()}_local_net', router=router3, node_count=1, cores=32, ram=128, disk=10)
    frr_experiment.add_chameleon_local_network(name=f'{router3.get_site()}_local_net', router=router3, node_count=1)
    
    print(f"router_names: {frr_experiment.router_names}")
    print(f"router_links: {frr_experiment.router_links}")
    print(f"local_networks: {frr_experiment.local_networks}")

   
    #frr_slice.save_config('config')   
    #frr_experiment.submit()
    
except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()


In [ ]:
try:
    frr_experiment.submit()
    
except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()

In [ ]:

#frr_experiment.load()
#print(f"{frr_experiment.nodes}")
#print(json.dumps(frr_experiment.nodes, indent=4))
frr_experiment.execute_on_all_edge_nodes(f'rm -rf fabric_node_tools')
frr_experiment.upload_directory_to_all_edge_nodes('fabric_node_tools','.')
frr_experiment.execute_on_all_edge_nodes(f'chmod +x fabric_node_tools/*.sh')





In [ ]:
try:
    #frr_slice = FRRouting_Slice(slice_name)
    
    frr_experiment.load()
except Exception as e:
    print(f"Exception: {e}")
    raise e

In [ ]:
try:
    frr_experiment.wait_jupyter()
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    frr_experiment.slice.save('ospf_slice.graphml')
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

In [ ]:

print(json.dumps(frr_experiment.local_networks, indent=4))


In [ ]:
import traceback
try:
    frr_experiment.configure_devs()
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

In [ ]:
import traceback
try:
    for node in frr_experiment.slice.get_nodes():
        print(f'{node.get_name()}: {node.get_management_ip()}')
        
        #node.execute('ip addr list', quiet=False)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

In [ ]:
import traceback
try:
    for node in frr_experiment.get_edge_nodes():
        print(f"node: {node.get_name()}")
        
    for router in frr_experiment.get_routers():
        print(f"router: {router.get_name()}")
        
    for local_networks in frr_experiment.get_local_networks():
        print(f"local_networks: {local_networks}")

    for router_links in frr_experiment.get_router_links():
        print(f"router_links: {router_links}")
 
    for local_network_name in frr_experiment.get_local_network_names():
        print(f"local_networks: {local_network_name}")

    for router_link_name in frr_experiment.get_router_link_names():
        print(f"router_links: {router_link_name}")
 
    
    frr_experiment.configure_routers()
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

In [ ]:
#frr_experiment.upload_directory_to_all_edge_nodes('fabric_node_tools','.')
command = f'sudo yum install -y -q iperf3 iproute-tc && sudo ./fabric_node_tools/host_tune_redhat.sh'


frr_experiment.execute_on_all_edge_nodes(f'{command}')

## Step 8: Run the Experiment

We will just test `ping` RTT and look at `tracepath`. Your experiment should be more interesting!

Notice that if you run this quickly and repeatedly run this test against a specific target, you may see changes to the tracepath.  Initially the ping may even fail.  Why do you think this is happening?


In [ ]:
try:
    target_node_name =  f'{site1}_local_net1'
    target_node = frr_experiment.slice.get_node(name=target_node_name)
    
    #print(f"{target_node.get_name()}")
    #target_iface = target_node.get_interface(network_name=f'{target_node.get_site()}_local_net')
    #print(f"{target_iface}")
    #target_iface_dev = target_iface.get_os_interface()
    #print(f"{target_iface_dev}")

    
    #target_ip = target_node.get_interface(network_name=f'{target_node.get_site()}_local_net').get_ips()
    target_ip = target_node.get_interface(network_name=f'{target_node.get_site()}_local_net').get_ip_addr()
    #print(f"{target_ip}")

    threads = {}
    for node in frr_experiment.slice.get_nodes():
        print(f"Testing target node: {node.get_name()}, target IP: {target_ip}")

        threads[node] = node.execute_thread(f'ping -c 5 {target_ip}')
        
    for node,thread in threads.items():
        print(f"Result for {node.get_name()}, target IP: {target_ip}")
        stdout, stderr = thread.result()
        print (stdout, stderr)

        #stdout, stderr = node.execute(f'tracepath {target_ip}')
        #print (stdout, stderr)

except Exception as e:
    print(f"Exception: {e}")

In [ ]:
for source_node in frr_experiment.nodes:
    for target_node in frr_experiment.nodes:
        
        if source_node['name'] == target_node['name']:
            continue
            
        frr_experiment.iperf3_run(source_node=source_node, 
                                  target_node=target_node, 
                                  w='32m', P=16, t=80, i=10, O=20, verbose=False)
    
    

In [ ]:
import traceback

try:
    iperf3_process_output(verbose=True)

except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Step 9: Delete the Slice

Please delete your slice when you are done with your experiment.

## Step 9: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    frr_experiment.fablib.delete_all()
    #slice.delete()
except Exception as e:
    print(f"Exception: {e}")